In [1]:
import fitsio
import numpy as np
#import healpy as hp
import pandas as pd
import matplotlib.pyplot as plt
#import b64encode
import mechanize
#from astropy import units as u
#from astropy.coordinates import SkyCoord, get_moon, EarthLocation
#from astropy.time import Time
#from astropy.io import fits
%pylab inline
%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
#loading file from csv sample
file_name = 'subset_spAll-v5_10_0.csv'
df_SKY=pd.read_csv(file_name)

#populating data frame
new_SKY = df_SKY.query('OBJTYPE=="SKY".ljust(16)')

#pulling arrays from data frames
DEC = np.array(new_SKY['DEC'].tolist())
FIBERID = np.array(new_SKY['FIBERID'].tolist())
MJD = np.array(new_SKY['MJD'].tolist())
RA = np.array(new_SKY['RA'].tolist())

#reading files
def read_flux_file(files):
#    files+='spectra/'
    fits = fitsio.FITS(files)
    cols = ['flux','loglam','ivar','and_mask','or_mask', 'sky', 'wdisp', 'model']
    d = {col: fits[1][col].read().byteswap().newbyteorder() for col in cols}
    df = pd.DataFrame(d)
    return df

#number of nodes in cluster
nodes=5
#file length/nodes (len(DEC)/nodes)
node_length=(100/nodes)

In [4]:
sum_nodes=[]
for n in range(nodes):
    node_part=[]
    for j in range(node_length):
        #loading in all files in range
        plates   = list(new_SKY['PLATE'][(j*n):(j*(n+1))].values)
        mjds   = list(new_SKY['MJD'][(j*n):(j*(n+1))].values)
        fiberids   = list(new_SKY['FIBERID'][(j*n):(j*(n+1))].values)
        
        #getting file names
        files = ['spec-%s-%s-%s.fits'%(plate,mjd,str(fiberid).zfill(4)) for plate,mjd,fiberid in zip(plates,mjds,fiberids)] #in cluster ['%s/spec...
        plate_n = ['%s'%(plate) for plate in plates]

        #reading files' loglam in
        dt_s=[]
        for l in files:
            dt_x = read_flux_file(l).set_index('loglam')
            dt_s.append(dt_x)
            
        # Masking
        mask=[]
        for s in dt_s:
            mask.append(s.query('and_mask==0').copy())
        #mask[1].head()
        
        #plot of each spectrum
#        fig = plt.figure(figsize = (15, 10))
#        xlimits = [3.55,4]
#        ylimits = [-10,20]
#        for i in range(len(files)):
#            dt_s[i]['flux'].plot(label=files[i],xlim=xlimits, ylim=ylimits)
#            dt_s[i]['sky'].plot(label=files[i],xlim=xlimits, ylim=ylimits)
#        plt.legend(loc='best')
        
#        print len(dt_s)
        for i in range(len(files)):
            dt_s[i]['flux_%s'%(i)] = dt_s[i]['flux']
            dt_s[i]['ivar_%s'%(i)] = dt_s[i]['ivar']
            dt_s[i]['sky_%s'%(i)] = dt_s[i]['sky']
                       
        x =range(len(files))
        result = pd.concat([dt_s[i][['flux_%s'%(i),'ivar_%s'%(i),'sky_%s'%(i)]] for i in x], axis=1)
        all_sky= result.fillna(0).copy()
        #all_sky.head()

0


ValueError: No objects to concatenate